In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from PIL import Image, ImageOps
from eidynamics import generate_camera_image
from eidynamics import pattern_index

In [ ]:
import pathlib
import importlib

In [ ]:
bfi = "C:\\users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Analysis\\testExamples\\slice_IR.bmp"
fli = "C:\\users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Analysis\\testExamples\\slice_tdTom.bmp"

In [ ]:
importlib.reload(generate_camera_image)
im = generate_camera_image.main(bfi,fluor_file=fli)
im.show()

In [ ]:
class Frame():
    def __init__(self, offset, channel='tdTomato'):
        self.LUT = {'tdTomato':(255,174,127,255),
                    'eYFP'   :(234,255,127,255),
                    'GFP'    :(127,255,127,255),
                    'IR'     :(200,200,200,255),
                    'grey'   :(200,200,200,255)}

        self.channel=channel
        self.channel_color = self.LUT[channel][:3]
        self.offset = offset
        self.mode   = ''
        self.alpha  = 1.0
        self.mask   = []

    def set_image(self,image):
        self.image  = image
        self.size   = self.image.size
        self.height,self.width = self.size
        self.mode   = self.image.mode

    def add_layer(self,new_layer_image,ofx,ofy):
        im1 = self.image
        im2 = new_layer_image
        # ofx = frameObj.offset[0]
        # ofy = frameObj.offset[1]
        new_image = im1.paste(im2,(ofx,ofy),mask=im2)
        return new_image

    def make_background_transparent(self,input_image,background_color=(0,0,0)):
        image = input_image.convert('RGBA')
        bgR,bgG,bgB = background_color
        # Transparency
        newImage = []
        for item in image.getdata():
            if item[:3] == background_color:
                new_color = (bgR,bgG,bgB,0)
                newImage.append(new_color)
            else:
                newImage.append(item)

        image.putdata(newImage)

        return image

    def apply_LUT(self,channel_color):
        im1 = self.image.convert('L')
        im2 = ImageOps.colorize(im1,black=(0,0,0),white=(255,255,255),mid=channel_color)
        return im2

    def set_mask():
        pass


In [ ]:
imageFile = "C:\\Users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Analysis\\testExamples\\slice1_tdTom.bmp"

In [ ]:
def pixel_scaling(x,m,c):
    y = m*np.array(x) + c
    return y

def calculate_offset(x=[0.25,0.5,0.75],y=[0.25,0.5,0.75],cx=[285,642,1002],cy=[341,535,730]):
    '''
    Data from 24 Dec 2021 Calibration
        Polygon Frame Fraction		Camera Pixel Number	
		x	    y	                cx	    cy
		0.25	0.25	            285	    341
		0.5	    0.5	                642	    535
		0.75	0.75	            1002	730
		0.125	0.125	            106	    244
		0.625	0.25	            822	    632
		0.375	0.375	            464	    439
		0.875	0.875	            1183	827
    '''
    poptx,_              = curve_fit(pixel_scaling,x,cx)
    popty,_              = curve_fit(pixel_scaling,y,cy)

    x0,y0 = [pixel_scaling(0,*poptx), pixel_scaling(0,*popty)]
    x1,y1 = [pixel_scaling(1,*poptx), pixel_scaling(1,*popty)]

    polygon_frame_properties =  {   
                                'top_left'      :[x0,y0],
                                'top_right'     :[x1,y0],
                                'bottom_right'  :[x0,y1],
                                'bottom_left'   :[x1,y1],
                                'width'         : x1-x0,
                                'height'        : y1-y0,
                                'aspect_ratio'  :(x1-x0) / (y1-y0),
                                'scaling'       :[round((x1-x0)/24), round((y1-y0)/24)],
                                'offsetx'       : int(x0),
                                'offsety'       : int(y0),
                                }

    return polygon_frame_properties

In [ ]:
def draw_polygon_frame(sqSet=[],grid_size=24,draw_grid=True):
    pixels = np.zeros((grid_size*grid_size,1))
    pixels[sqSet] = 1
    frameArray = np.reshape(pixels,(grid_size,grid_size))
    frameImage = Image.fromarray(frameArray*255).convert('RGBA')

    frame_properties = calculate_offset()
    scaling = frame_properties['scaling']
    frameLarge = frameImage.resize(np.multiply(frameImage.size,scaling),0)

    if draw_grid:
        grid = np.zeros(frameLarge.size).T
        grid[:,::scaling[0]] = 1
        grid[::scaling[1],:] = 1
        gridImage = Image.fromarray(grid*255).convert('RGBA')

        polygon_frame_image = Image.blend(frameLarge,gridImage,0.3)
    else:
        polygon_frame_image = frameLarge

    output_image = replace_color_with_transparency(polygon_frame_image)

    return output_image,frame_properties


In [ ]:
def replace_color_with_transparency(input_image,remove_color=(0,0,0)):
    image = input_image.convert('RGBA')

    # Transparency
    newImage = []
    for item in image.getdata():
        if item[:3] == remove_color:
            new_color = (remove_color[0],remove_color[1],remove_color[2],0)
            newImage.append(new_color)
        else:
            newImage.append(item)

    image.putdata(newImage)

    return image
    

In [ ]:
camframe = Frame(offset=[0,0],channel='IR')

In [ ]:
camfile = "C:\\Users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Data\\21-12-31_G620\\6201_CA3.bmp"

In [ ]:
# with Image.open(camfile,'r') as c:
camImage = Image.open(camfile)
camframe.set_image(camImage)

In [ ]:
polyImage,polyImProp = draw_polygon_frame(sqSet=pattern_index.patternID[999])

In [ ]:
polyframe = Frame(offset=[polyImProp['offsetx'],polyImProp['offsety']],channel='GFP')

In [ ]:
polyframe.set_image(polyImage)
# newImage = polyframe.apply_LUT(polyframe.channel_color)

In [ ]:
newImage = polyframe.apply_LUT(polyframe.channel_color)

In [ ]:
newImage2 = polyframe.make_background_transparent(newImage)

In [ ]:
image3 = camframe.add_layer(newImage2,polyframe.offset[0],polyframe.offset[1])

In [ ]:
camframe.image

In [ ]:
def add_offset(input_image, left, top, right, bottom, color=(0,0,0)):
    width, height = input_image.size
    new_width = width + right + left
    new_height = height + top + bottom
    canvas = Image.new(input_image.mode, (new_width, new_height), color)
    canvas.paste(input_image, (left, top))
    return canvas

In [ ]:
im1 = camImage.copy().convert('RGBA')
maask = newImage2.getchannel('A')
im1.paste(newImage2,(-75,146),mask=newImage2)

In [ ]:
camImage